### 1. Import Libraries

In [1]:
import mmcv
import matplotlib.pyplot as plt

import os.path as osp
import numpy as np
from PIL import Image
import os
import sys
import ade_custom
import dataset_split as ds
import matplotlib.patches as mpatches
import random

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmseg
print(mmseg.__version__)

/root/miniconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.1 True
0.28.0


### 2. Add a new dataset
data_root, img_dir, ann_dir 변경

In [2]:

import shutil
import random
from glob import glob
import json

# data 경로 설정
data_root = '/data/36-3/'
img_path = data_root + 'img_dir/'
ann_path = data_root + 'ann_dir/'
img_paths = glob(img_path+'*')
ann_paths = glob(ann_path+'*')
img_move_path = data_root + 'img_val/'
ann_move_path = data_root + 'ann_val/'
img_move_path_t = data_root + 'img_test/'
ann_move_path_t = data_root + 'ann_test/'
label_path = '/data/36-3/label_data/'
label_folder = glob(label_path+'*')
ds.root = data_root
ds.label_path = label_path
# new dataset classes, class별 색상
classes = ade_custom.COLOR_PARAM.CLASSES
palette = ade_custom.COLOR_PARAM.COLORMAP

### 2-1. Check Image Data

In [3]:
# # # Let's take a look at the segmentation map we got
# img = Image.open('/mmsegmentation/data/36-3/ann_dir/24_193937_220616_170.png')
# plt.figure(figsize=(8, 6))
# im = plt.imshow(np.array(img.convert('RGB')))

# # create a patch (proxy artist) for every color 
# patches = [mpatches.Patch(color=np.array(palette[i])/255., 
#                           label=classes[i]) for i in range(8)]
# # put those patched as legend-handles into the legend
# plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
#            fontsize='large')

# plt.show()

### 3. Split dataset randomly

In [4]:
# split 한 데이터 다시 원래 이미지경로로 복원
ds.restore_split(img_path, ann_path,img_move_path, ann_move_path, img_move_path_t, ann_move_path_t)

restore Done!


In [5]:
classes, file_list = ds.make_label_list(label_folder)
classes = ds.dict_key_lower(classes)
file_list = ds.dict_key_lower(file_list)

In [6]:
ds.s_label_split(img_path, ann_path, img_move_path, ann_move_path, img_move_path_t, ann_move_path_t, classes, file_list)
ds.label_split(img_path, ann_path, img_move_path, ann_move_path, img_move_path_t, ann_move_path_t)

s_label_split finish!
label_split finish!


### 3-1. Train Image Normalize

In [7]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import cv2

class LeafData(Dataset):
    
    def __init__(self, 
                 data, 
                 transform = None):
        self.data      = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        # import
        path  = self.data[idx]
        img_array = np.fromfile(path, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
        # augmentations
        if self.transform is not None:
            image = self.transform(image = image)['image']
        
        return image

In [8]:
augs = A.Compose([A.Resize(height = 288, 
                           width  = 512), 
                  A.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1)),
                  ToTensorV2()])

In [9]:
# dataset
img_paths = glob(img_path+'*')
image_dataset = LeafData(data      = img_paths, 
                         transform = augs)

# data loader
image_loader = DataLoader(image_dataset, 
                          batch_size  = 8, 
                          shuffle     = False, 
                          num_workers = 4,
                          pin_memory  = True)

In [10]:
# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████| 855/855 [01:07<00:00, 12.70it/s]


In [11]:
# pixel count
count = len(img_paths) * 288 * 512

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

In [12]:
means = np.round(np.array(total_mean * 256),2)
stds = np.round(np.array(total_std * 256),2)

# output
print('mean: '  + str(means))
print('std:  '  + str(stds))

mean: [108.09 109.48 113.55]
std:  [46.29 47.66 54.99]


In [13]:
print(len(glob(data_root + 'img_dir/*')))
print(len(glob(data_root + 'img_val/*')))
print(len(glob(data_root + 'img_test/*')))

print(len(glob(data_root + 'ann_dir/*')))
print(len(glob(data_root + 'ann_val/*')))
print(len(glob(data_root + 'ann_test/*')))

6835
839
846
6715
839
846


In [14]:
# train:valid:test / 8:1:1 비율로 split
split_dir = 'splits'
mmcv.mkdir_or_exist(osp.join(data_root, split_dir))
origin_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_path), suffix='.png')]
valid_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_move_path), suffix='.png')]
test_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, ann_move_path_t), suffix='.png')]

random.shuffle(origin_list)
with open(osp.join(data_root, split_dir, 'train.txt'), 'w') as f:
  f.writelines(line + '\n' for line in origin_list)

random.shuffle(valid_list)
with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
  f.writelines(line + '\n' for line in valid_list)

random.shuffle(test_list)
with open(osp.join(data_root, split_dir, 'test.txt'), 'w') as f:
  f.writelines(line + '\n' for line in test_list)

# with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
#   val_length = int(len(filename_list)*9/10)
#   f.writelines(line + '\n' for line in filename_list[train_length:val_length])

# with open(osp.join(data_root, split_dir, 'test.txt'), 'w') as f:
#   f.writelines(line + '\n' for line in filename_list[val_length:])

### 4. Pipeline

In [15]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset


In [16]:
from mmcv import Config
# cfg = Config.fromfile('/home/mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py')

# cfg = Config.fromfile('/home/mmsegmentation/custom_config_train.py')

cfg = Config.fromfile('/home/mmsegmentation/custom_config_train_3.py')

In [17]:
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 32
cfg.model.auxiliary_head.num_classes = 32

# Modify dataset type and path
cfg.dataset_type = 'CustomDataset'
cfg.data_root = '/data/36-3/'
# batch_size
cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu= 4

cfg.img_norm_cfg = dict(
    # 36-3 mean,std
      mean=[means[0], means[1], means[2]],std=[stds[0], stds[1], stds[2]], to_rgb=True)
#cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label = False),
    dict(type='Resize', img_scale=(512, 512)),
    #dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    #dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
 
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_path
cfg.data.train.ann_dir = ann_path
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = data_root+'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_move_path
cfg.data.val.ann_dir = ann_move_path
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = data_root+'splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_move_path_t
cfg.data.test.ann_dir = ann_move_path_t
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = data_root+'splits/test.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth'

# cfg.load_from = 'data/result/36-3/test0913/iter_1600.pth'
# Set up working dir to save files and logs.
cfg.work_dir = '/data/result/36-3/221011/'

cfg.runner.max_iters = 160000
cfg.log_config.interval = 1600
cfg.evaluation.interval = 8000
cfg.checkpoint_config.interval = 16000

# Set seed to facitate reproducing the result
cfg.seed = 42
set_random_seed(42, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=384,
        embed_dims=128,
        patch_size=4,
        window_size=12,
        mlp_ratio=4,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            '/home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth'
        )),
    decode_head=dict(
        type='UPerHead',
        in_channels=[128, 256, 5

In [18]:
# from mmseg.apis import set_random_seed
# from mmseg.utils import get_device

# # Modify dataset type and path
# cfg.dataset_type = 'ADE20KDataset'
# cfg.data_root = '/mmsegmentation/data/36-3/'

# # We can still use the pre-trained Mask RCNN model though we do not need to
# # use the mask branch
# # cfg.load_from = '/mmsegmentation/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K_20210531_125459-429057bf.pth'

# cfg.load_from = '/mmsegmentation/data/result/36-3/test0923/latest.pth'
# # Set up working dir to save files and logs.
# cfg.work_dir = '/mmsegmentation/data/result/36-3/test0927/'


# # Set seed to facitate reproducing the result
# cfg.seed = 42
# set_random_seed(42, deterministic=False)
# cfg.gpu_ids = range(1)
# cfg.device = get_device()

# # Let's have a look at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

### 6. Train and Evaluation

In [19]:
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Clear Cache
torch.cuda.empty_cache()

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# print(datasets[0][0]['img'].__dict__)
#print(datasets[0][0])

# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
print(len(model.CLASSES))
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2022-10-12 11:41:22,083 - mmseg - INFO - Loaded 6715 images
2022-10-12 11:41:23,111 - mmseg - INFO - Loaded 839 images
2022-10-12 11:41:23,112 - mmseg - INFO - load checkpoint from local path: /home/mmsegmentation/checkpoints/swin_base_patch4_window12_384_22k.pth
2022-10-12 11:41:23,195 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: model

missing keys in source state_dict: backbone.patch_embed.projection.weight, backbone.patch_embed.projection.bias, backbone.patch_embed.norm.weight, backbone.patch_embed.norm.bias, backbone.stages.0.blocks.0.norm1.weight, backbone.stages.0.blocks.0.norm1.bias, backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.0.attn.w_msa.qkv.weight, backbone.stages.0.blocks.0.attn.w_msa.qkv.bias, backbone.stages.0.blocks.0.attn.w_msa.proj.weight, backbone.stages.0.blocks.0.attn.w_msa.proj.bias, backbo

32


2022-10-12 11:55:32,453 - mmseg - INFO - Iter [1600/160000]	lr: 5.940e-05, eta: 23:20:57, time: 0.531, data_time: 0.007, memory: 12927, decode.loss_ce: 0.7384, decode.acc_seg: 87.4877, aux.loss_ce: 0.4698, aux.acc_seg: 82.1689, loss: 1.2082
2022-10-12 12:09:42,175 - mmseg - INFO - Iter [3200/160000]	lr: 5.880e-05, eta: 23:07:19, time: 0.531, data_time: 0.009, memory: 12927, decode.loss_ce: 0.1330, decode.acc_seg: 95.9688, aux.loss_ce: 0.0897, aux.acc_seg: 93.9635, loss: 0.2227
2022-10-12 12:23:51,629 - mmseg - INFO - Iter [4800/160000]	lr: 5.820e-05, eta: 22:53:12, time: 0.531, data_time: 0.009, memory: 12927, decode.loss_ce: 0.0912, decode.acc_seg: 97.0083, aux.loss_ce: 0.0624, aux.acc_seg: 95.1590, loss: 0.1536
2022-10-12 12:38:01,687 - mmseg - INFO - Iter [6400/160000]	lr: 5.760e-05, eta: 22:39:18, time: 0.531, data_time: 0.009, memory: 12927, decode.loss_ce: 0.0748, decode.acc_seg: 97.4542, aux.loss_ce: 0.0521, aux.acc_seg: 95.7677, loss: 0.1268
2022-10-12 12:52:11,424 - mmseg - IN

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 839/839, 16.7 task/s, elapsed: 50s, ETA:     0s

2022-10-12 12:53:01,818 - mmseg - INFO - per class results:
2022-10-12 12:53:01,821 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 98.45 | 99.46 |
|      vehicle      | 78.45 | 93.33 |
|        bus        |  78.3 | 85.54 |
|       truck       | 68.31 | 74.36 |
|     policeCar     |  nan  |  nan  |
|     ambulance     |  nan  |  nan  |
|     schoolBus     | 40.65 | 41.68 |
|      otherCar     | 63.21 | 83.53 |
|     motorcycle    |  0.0  |  0.0  |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  nan  |  nan  |
|     pedestrian    |  0.0  |  0.0  |
|       rider       |  0.0  |  0.0  |
|     freespace     | 91.97 | 95.84 |
|        curb       | 71.13 | 81.99 |
|      sidewalk     |  0.0  |  0.0  |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    | 83.59 | 89.95 |
|     speedBump     |  nan  |  nan  |
|      roadMark     | 27.91 | 31.53 |
|     whiteLane     | 45

...

2022-10-12 16:28:08,320 - mmseg - INFO - per class results:
2022-10-12 16:28:08,321 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 98.99 | 99.63 |
|      vehicle      |  89.2 | 94.59 |
|        bus        | 89.58 |  93.3 |
|       truck       | 86.05 | 91.36 |
|     policeCar     |  nan  |  nan  |
|     ambulance     |  nan  |  nan  |
|     schoolBus     | 46.34 | 47.05 |
|      otherCar     | 75.99 | 90.95 |
|     motorcycle    |  0.0  |  0.0  |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  nan  |  nan  |
|     pedestrian    | 53.77 | 73.46 |
|       rider       |  0.0  |  0.0  |
|     freespace     |  93.6 | 96.97 |
|        curb       | 79.26 | 88.75 |
|      sidewalk     |  1.1  |  1.19 |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    | 88.55 | 93.86 |
|     speedBump     |  nan  |  nan  |
|      roadMark     | 43.77 | 50.69 |
|     whiteLane     | 50

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 839/839, 16.5 task/s, elapsed: 51s, ETA:     0s

2022-10-12 17:39:49,398 - mmseg - INFO - per class results:
2022-10-12 17:39:49,400 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 99.01 | 99.62 |
|      vehicle      | 89.96 | 94.55 |
|        bus        |  90.5 | 94.13 |
|       truck       | 87.52 | 91.83 |
|     policeCar     |  nan  |  nan  |
|     ambulance     |  nan  |  nan  |
|     schoolBus     | 47.61 |  48.4 |
|      otherCar     | 77.95 | 91.93 |
|     motorcycle    |  0.0  |  0.0  |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  nan  |  nan  |
|     pedestrian    | 52.53 | 71.13 |
|       rider       |  0.0  |  0.0  |
|     freespace     | 93.92 | 96.94 |
|        curb       | 80.92 | 89.75 |
|      sidewalk     | 65.68 | 74.09 |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    | 89.71 | 95.23 |
|     speedBump     |  nan  |  nan  |
|      roadMark     |  46.4 | 55.25 |
|     whiteLane     | 54

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 839/839, 16.6 task/s, elapsed: 50s, ETA:     0s

2022-10-12 18:51:34,974 - mmseg - INFO - per class results:
2022-10-12 18:51:34,976 - mmseg - INFO - 
+-------------------+-------+-------+
|       Class       |  IoU  |  Acc  |
+-------------------+-------+-------+
|     background    | 99.01 | 99.63 |
|      vehicle      | 89.81 | 94.81 |
|        bus        | 89.98 | 93.32 |
|       truck       | 84.43 | 89.13 |
|     policeCar     |  nan  |  nan  |
|     ambulance     |  nan  |  nan  |
|     schoolBus     | 43.92 | 44.53 |
|      otherCar     | 77.35 | 92.06 |
|     motorcycle    |  0.0  |  0.0  |
|      bicycle      |  0.0  |  0.0  |
|     twoWheeler    |  nan  |  nan  |
|     pedestrian    | 56.46 | 63.12 |
|       rider       |  0.0  |  0.0  |
|     freespace     | 93.83 | 97.08 |
|        curb       | 79.54 | 89.48 |
|      sidewalk     | 60.54 | 65.57 |
|     crossWalk     |  0.0  |  0.0  |
|     safetyZone    | 89.78 | 93.95 |
|     speedBump     |  nan  |  nan  |
|      roadMark     | 44.94 | 52.02 |
|     whiteLane     | 53

In [ ]:
palette = [
                [0,0,0],[128, 0, 0], [0, 128, 0], [0, 0, 128], [128, 128, 0],  [128, 0, 128], [0, 128, 128], [128, 128, 128], 
                [64, 0, 0], [0, 64, 0], [0, 0, 64], [64, 64, 0],  [64, 0, 64], [0, 64, 64], [64, 64, 64], 
                [192, 0, 0], [0, 192, 0], [0, 0, 192], [192, 192, 0],  [192, 0, 192], [0, 192, 192], [192, 192, 192], 
                [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128], [64, 128, 128], 
                [192, 128, 128], [128, 64, 0], [0, 192, 128], [128, 192, 0], [0, 64, 128]
                ]

In [ ]:
import ade_custom
model = torch.load(cfg.work_dir+'latest.pth')
model['meta']['PALETTE'] = palette
torch.save(model, cfg.work_dir+'latest_fix.pth')

### 7. Inference with trained model

In [ ]:
config_file = '/home/mmsegmentation/custom_config_test.py'
checkpoint_file = '/data/result/36-3/test0928_2/latest_fix.pth'

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

img = '/data/36-3/img_test/16_142441_220613_150.jpg'

result = inference_segmentor(model, img)

show_result_pyplot(model, img, result, palette)